# Experimento - Tempo

Neste código está o experimento proposto para a análise de tempo apresentada na Seção 3.1.5

In [ ]:
''' Bibliotecas Necessárias '''
import numpy as np
import pandas as pd
import time
from IPython.display import clear_output
''' Código que chama o MIRT para a realização do experimento '''
from mirt import Mirt
''' Define a semente '''
np.random.seed(42)

In [ ]:
def generate_answers(n):
    '''
    Função que gera um conjunto de respostas aleatórias para um item (dado pela probabilidade p de acerto).
    Entradas:
        n = número de indivíduos
    '''
    p = np.random.rand()
    return np.random.choice([0, 1], size=n, p=[1.0-p, p])

def generate_test(n, t):
    '''
    Função que gera um teste, onde cada item do teste é gerado pela fuunção generate_answers
    Entradas: 
        n = número de indivíduos
        t = número de itens
    '''
    test = pd.DataFrame()
    for i in range(0, t):
        ctrl = 1
        # se houver ao menos um erro e um acerto, salva o resultado, caso contrário gera de novo
        while ctrl == 1:
            value = generate_answers(n)
            ctrl = len(np.unique(value)) 
        test[i] = value 
    return test

def time_item_experiment(N, T, q):
    '''
    Função que gera um experimento completo de análise de tempo
    Entradas: 
        N = vetor com a variação de indivíduos a ser testada
        T = vetor com a variação de itens a ser testada
        q = número de vezes que repete a mesma configuração
    '''
    report = pd.DataFrame(columns = ['TRI', 'N_ID', 'N_ITENS', 'TIME'])
    total = len(N)*len(T)*q*3
    count = 0
    # Varia o número de individuos
    for n in N:
        # Varia o número de itens
        for t in T:
            # Repete o exemperimento q vezes
            for i in range(0, q):
                test = generate_test(n, t)
                # Varia os modelos
                for model in ['Rasch', '2PL', '3PL']:
                    clear_output()
                    print(("%d/%d") % (count, total))
                    inicio = time.time()
                    mirt_model = Mirt()
                    mirt_model.calculate(test, model)
                    fim = time.time()
                    report.loc[count] = [model, n, t, fim - inicio]
                    count += 1

    return report

In [ ]:
'''
---
Experimento 1 - Análise dos Itens
---
Fixa o número de indivíduos em 5000 e varia o número de itens de 5 a 250.
''' 
N = [5000]
T = np.arange(5, 250, 5)
rel1 = time_item_experiment(N, T, 10)
rel1.to_csv('exp1.csv', sep=';', index=False)

In [ ]:
'''
---
Experimento 2 - Análise dos Indivíduos
---
Fixa o número de itens em 50 e varia o número de indivíduos de 10 a 40000.
'''
N = np.arange(10, 40210, 200)
T = [50]
rel2 = time_item_experiment(N, T, 10)
rel2.to_csv('exp2.csv', sep=';', index=False)